In [1]:
import pandas as pd

import common

In [2]:
df = pd.read_csv(common.FINAL_PROFILE_DATABASE)

In [3]:
if df.nvcc_version.unique().shape[0] == 1:
    df = df.drop(columns="nvcc_version")

In [4]:
df

,board,app,hardening,family,config,execution_time,cf_executed,inst_executed,achieved_occupancy,ipc,ldst_fu_utilization,sm_efficiency,static_shared_per_block_max,dynamic_shared_per_block_max,rf_max,active_cycles_pm,elapsed_cycles_pm,elapsed_cycles_sm,shared_max
0,pascal,vit_base_patch16_224,replace-id,Original,B16-224,0.1399,4.741263e+07,4.130818e+09,0.927877,3.732211,Low (2),99.662572%,16384.0,21248.0,163.0,2.283647e+09,2.322108e+09,2.337840e+09,NaN
1,pascal,vit_base_patch16_224,no-replace-id,Original,B16-224,0.1351,3.363833e+07,4.054414e+09,0.919298,3.674832,Low (2),99.619861%,16384.0,21248.0,163.0,2.215014e+09,2.248737e+09,2.257987e+09,NaN
2,pascal,vit_base_patch16_384,replace-id,Original,B16-384,0.4723,1.155875e+08,1.201321e+10,0.935054,3.914451,Low (2),99.882540%,16384.0,21248.0,163.0,7.927851e+09,7.997507e+09,7.859772e+09,NaN
3,pascal,vit_base_patch16_384,no-replace-id,Original,B16-384,0.4607,7.523273e+07,1.178940e+10,0.934730,3.909180,Low (2),99.908796%,16384.0,21248.0,163.0,7.593302e+09,7.656862e+09,7.796461e+09,NaN
4,pascal,vit_base_patch32_224.sam,replace-id,Original,BS32-224,0.0321,9.921411e+06,9.778978e+08,0.880115,3.297002,Low (2),99.134761%,16384.0,21248.0,163.0,4.282240e+08,4.700102e+08,4.658406e+08,NaN
5,pascal,vit_base_patch32_224.sam,no-replace-id,Original,BS32-224,0.0313,6.433754e+06,9.585290e+08,0.876469,2.876548,Low (2),99.452756%,16384.0,21248.0,163.0,4.222693e+08,4.591082e+08,4.573109e+08,NaN
6,pascal,vit_large_patch14_clip_224.laion2b_ft_in12k_in1k,replace-id,Original,L14-224,0.5056,1.235965e+08,1.733898e+10,0.931753,3.826081,Low (3),99.810886%,16384.0,21248.0,163.0,8.106910e+09,8.332566e+09,8.385819e+09,NaN
7,pascal,vit_large_patch14_clip_224.laion2b_ft_in12k_in1k,no-replace-id,Original,L14-224,0.4881,7.647263e+07,1.707681e+10,0.932127,3.820848,Low (3),99.622878%,16384.0,21248.0,163.0,7.875923e+09,8.093991e+09,8.020378e+09,NaN
8,pascal,vit_huge_patch14_clip_224.laion2b_ft_in12k_in1k,replace-id,Original,H14-224,1.6725,2.096146e+08,3.575576e+10,0.936286,3.815195,Low (2),99.861017%,16384.0,24960.0,168.0,2.788106e+10,2.832371e+10,2.821664e+10,NaN
9,pascal,vit_huge_patch14_clip_224.laion2b_ft_in12k_in1k,no-replace-id,Original,H14-224,1.6440,1.311605e+08,3.531927e+10,0.933149,3.805617,Low (2),99.857414%,16384.0,24960.0,168.0,2.746958e+10,2.788616e+10,2.785703e+10,NaN


In [5]:
grouped = df.set_index(["board", "family", "config", "hardening"]).drop(columns=["app"])

In [6]:
hardened = grouped.loc[pd.IndexSlice[:, :, :, "replace-id"]].droplevel("hardening")
unhardened = grouped.loc[pd.IndexSlice[:, :, :, "no-replace-id"]].droplevel("hardening")

## Calculate the diff

In [7]:
diff = pd.concat(
    [
        (hardened["cf_executed"] - unhardened["cf_executed"]) / ((hardened["cf_executed"] + unhardened["cf_executed"]) / 2),
        (hardened["inst_executed"] - unhardened["inst_executed"]) / ((hardened["inst_executed"] + unhardened["inst_executed"]) / 2),
        (hardened["elapsed_cycles_sm"] - unhardened["elapsed_cycles_sm"]) / ((hardened["elapsed_cycles_sm"] + unhardened["elapsed_cycles_sm"]) / 2),
        (hardened["execution_time"] - unhardened["execution_time"]) / ((hardened["execution_time"] + unhardened["execution_time"]) / 2)
    ],
    axis=1
)

In [8]:
diff

cf_executed  inst_executed  elapsed_cycles_sm  \
board  family   config                                                    
pascal Original B16-224      0.339892       0.018669           0.034750   
                B16-384      0.422961       0.018806           0.008088   
                BS32-224     0.426490       0.020005           0.018480   
                L14-224      0.471076       0.015235           0.044549   
                H14-224      0.460445       0.012283           0.012826   
       SwinV2   B-256        0.005133       0.000895           0.005970   
                B-384        0.003494       0.000672          -0.003857   
                L-256        0.005078       0.000711          -0.000590   
                L-384        0.003525       0.000571           0.003913   
       EVA      B14-448      0.093382       0.005413           0.022854   
                L14-448      0.085165       0.004268           0.002644   
                S14-336      0.143095       0.015207           0.051204   
       MaxViT   L-384        0.178968       0.027620           0.133925   
                L-512        0.166663       0.027100           0.124624   
ampere Original B16-224      0.293621       0.017084           0.083799   
                B16-384      0.307481       0.016887           0.072750   
                BS32-224     0.279540       0.017192           0.102174   
                L14-224      0.254950       0.012678           0.060506   
                H14-224      0.286676       0.010506           0.048375   
       SwinV2   B-256        0.005289       0.000961           0.004674   
                B-384        0.003468       0.000691           0.004085   
                L-256        0.005406       0.000756           0.004167   
                L-384        0.003521       0.000578           0.003290   
       EVA      B14-448      0.086301       0.005744           0.029568   
                L14-448      0.077644       0.004387           0.024334   
                S14-336      0.165699       0.018114           0.085005   
       MaxViT   L-384        0.170703       0.031033           0.142673   
                L-512        0.162396       0.031879           0.140355   

                          execution_time  
board  family   config                    
pascal Original B16-224         0.034909  
                B16-384         0.024866  
                BS32-224        0.025237  
                L14-224         0.035222  
                H14-224         0.017187  
       SwinV2   B-256           0.005470  
                B-384           0.004655  
                L-256           0.002471  
                L-384           0.000307  
       EVA      B14-448         0.015762  
                L14-448         0.013682  
                S14-336         0.053265  
       MaxViT   L-384           0.107781  
                L-512           0.095115  
ampere Original B16-224         0.100604  
                B16-384         0.091939  
                BS32-224        0.072993  
                L14-224         0.064989  
                H14-224         0.051328  
       SwinV2   B-256           0.005644  
                B-384           0.003476  
                L-256           0.008658  
                L-384           0.005536  
       EVA      B14-448         0.032327  
                L14-448         0.023091  
                S14-336         0.094926  
       MaxViT   L-384           0.160927  
                L-512           0.153078

In [9]:
grouped

execution_time   cf_executed  \
board  family   config   hardening                                     
pascal Original B16-224  replace-id             0.1399  4.741263e+07   
                         no-replace-id          0.1351  3.363833e+07   
                B16-384  replace-id             0.4723  1.155875e+08   
                         no-replace-id          0.4607  7.523273e+07   
                BS32-224 replace-id             0.0321  9.921411e+06   
                         no-replace-id          0.0313  6.433754e+06   
                L14-224  replace-id             0.5056  1.235965e+08   
                         no-replace-id          0.4881  7.647263e+07   
                H14-224  replace-id             1.6725  2.096146e+08   
                         no-replace-id          1.6440  1.311605e+08   
       SwinV2   B-256    replace-id             0.1833  3.004139e+08   
                         no-replace-id          0.1823  2.988759e+08   
                B-384    replace-id             0.5814  9.913350e+08   
                         no-replace-id          0.5787  9.878770e+08   
                L-256    replace-id             0.4052  4.553428e+08   
                         no-replace-id          0.4042  4.530363e+08   
                L-384    replace-id             1.3049  1.473799e+09   
                         no-replace-id          1.3045  1.468612e+09   
       EVA      B14-448  replace-id             0.8249  3.280067e+08   
                         no-replace-id          0.8120  2.987430e+08   
                L14-448  replace-id             2.7227  9.284763e+08   
                         no-replace-id          2.6857  8.526319e+08   
                S14-336  replace-id             0.1195  1.134225e+08   
                         no-replace-id          0.1133  9.827602e+07   
       MaxViT   L-384    replace-id             1.0443  1.550758e+09   
                         no-replace-id          0.9375  1.296016e+09   
                L-512    replace-id             1.9384  2.943711e+09   
                         no-replace-id          1.7624  2.490841e+09   
ampere Original B16-224  replace-id             0.0261  2.836066e+07   
                         no-replace-id          0.0236  2.109940e+07   
                B16-384  replace-id             0.0785  7.977451e+07   
                         no-replace-id          0.0716  5.851399e+07   
                BS32-224 replace-id             0.0071  7.511728e+06   
                         no-replace-id          0.0066  5.669404e+06   
                L14-224  replace-id             0.1128  1.097458e+08   
                         no-replace-id          0.1057  8.492962e+07   
                H14-224  replace-id             0.2278  1.647521e+08   
                         no-replace-id          0.2164  1.234428e+08   
       SwinV2   B-256    replace-id             0.0533  1.632735e+08   
                         no-replace-id          0.0530  1.624121e+08   
                B-384    replace-id             0.1729  5.594320e+08   
                         no-replace-id          0.1723  5.574955e+08   
                L-256    replace-id             0.0928  2.395830e+08   
                         no-replace-id          0.0920  2.382913e+08   
                L-384    replace-id             0.2898  8.264018e+08   
                         no-replace-id          0.2882  8.234973e+08   
       EVA      B14-448  replace-id             0.1666  1.977592e+08   
                         no-replace-id          0.1613  1.813984e+08   
                L14-448  replace-id             0.5213  5.682883e+08   
                         no-replace-id          0.5094  5.258131e+08   
                S14-336  replace-id             0.0320  5.544120e+07   
                         no-replace-id          0.0291  4.695750e+07   
       MaxViT   L-384    replace-id             0.3310  9.070258e+08   
                         no-replace-id          0.2817  7.643701e+08   
        

In [15]:
family_average_diff = diff.reset_index().groupby(["board", "family"]).mean()
family_average_diff_stddev = diff.reset_index().groupby(["board", "family"]).std()

In [11]:
family_average_diff

cf_executed  inst_executed  elapsed_cycles_sm  execution_time
board  family                                                                 
ampere EVA          0.109881       0.009415           0.046302        0.050115
       MaxViT       0.166549       0.031456           0.141514        0.157003
       Original     0.284453       0.014869           0.073520        0.076370
       SwinV2       0.004421       0.000746           0.004054        0.005829
pascal EVA          0.107214       0.008296           0.025567        0.027570
       MaxViT       0.172816       0.027360           0.129274        0.101448
       Original     0.424173       0.016999           0.023739        0.027484
       SwinV2       0.004308       0.000712           0.001359        0.003226

In [20]:
with pd.ExcelWriter("/home/fernando/git_research/date_2024_vits/data/metrics_tmp.xlsx") as writer:
    grouped.to_excel(writer, sheet_name="raw")
    unhardened.to_excel(writer, sheet_name="raw_unhard")
    diff.to_excel(writer, sheet_name="diff%")
    family_average_diff.to_excel(writer, sheet_name="fam_avg_diff")
    family_average_diff.unstack(level=0).to_excel(writer, sheet_name="fam_avg_diff_T")
    family_average_diff_stddev.to_excel(writer, sheet_name="fam_avg_diff_std")